In [14]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
import torch
from utilities import TermDepositData,BinaryClassification
from torchmetrics.functional import accuracy

## Making the tensor object from csv data

In [15]:
final_train= pd.read_csv('preprocessed_data/final_train.csv')
final_test = pd.read_csv('preprocessed_data/final_test.csv')
column_names = final_train.columns 
selected_labels = ['balance','day','pdays','duration','campaign','previous','age','housing_yes','loan_yes','target']
final_train = final_train[selected_labels]
final_test = final_test[selected_labels]

target_label = 'target'
feature_labels = [i for i in selected_labels if i not in [target_label]]

train_tensor = torch.FloatTensor(final_train.values)
test_tensor = torch.FloatTensor(final_test.values)

In [16]:
# train_tensor = torch.load('torch_objects/train_tensor.pt')
# test_tensor = torch.load('torch_objects/test_tensor.pt')

In [17]:
train_tensor.shape, test_tensor.shape

(torch.Size([45211, 10]), torch.Size([4521, 10]))

## Setting the Dataloader and DataClass

In [18]:
training_data = TermDepositData(train_tensor,selected_labels)
testing_data = TermDepositData(test_tensor,selected_labels)

In [19]:
# training and testing length 
training_data.__len__(),testing_data.__len__()

(45211, 4521)

In [20]:
training_data[0]# input, ouptut 

(tensor([ 2.1430e+03,  5.0000e+00, -1.0000e+00,  2.6100e+02,  1.0000e+00,
          0.0000e+00,  5.8000e+01,  1.0000e+00,  0.0000e+00]),
 tensor(0.))

In [21]:
train_dataloader = torch.utils.data.DataLoader(
    training_data,batch_size =128
)
test_dataloader = torch.utils.data.DataLoader(
    testing_data,batch_size =128
)
feature_dim = test_dataloader.dataset.feature_names.__len__()

## Model Initialization 

In [22]:
model = BinaryClassification()
model

BinaryClassification(
  (layer_1): Linear(in_features=9, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

### Initial Weights of Model 

In [23]:
for i in model.parameters():
    print(i.shape)  # weight matrices

torch.Size([64, 9])
torch.Size([64])
torch.Size([64, 64])
torch.Size([64])
torch.Size([1, 64])
torch.Size([1])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])


In [33]:
learning_rate = 0.01
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 10

In [34]:
def train_loop(model,epochs,dataloader,optimizer,loss_fn,accuracy_metric):
    model.train()
    for e in range(1, epochs+1):
        epoch_loss = 0
        epoch_acc = 0
        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            
            loss = loss_fn(y_pred, y_batch.unsqueeze(1))
            acc = accuracy(y_pred,y_batch.int())

            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
        loss_value = epoch_loss/len(dataloader)
        acc_value  = epoch_acc/len(dataloader)
        print(f'Epoch {e+0:02}: | Loss: {loss_value:.5f} | Acc: {acc_value:.3f}')

In [35]:
train_loop(model,epochs,train_dataloader,optimizer,loss_fn,accuracy)

Epoch 01: | Loss: 207.05556 | Acc: 0.866
Epoch 02: | Loss: 171.54800 | Acc: 0.844
Epoch 03: | Loss: 130.69717 | Acc: 0.844
Epoch 04: | Loss: 98.35253 | Acc: 0.845
Epoch 05: | Loss: 81.19551 | Acc: 0.849
Epoch 06: | Loss: 70.40544 | Acc: 0.848
Epoch 07: | Loss: 67.61496 | Acc: 0.846
Epoch 08: | Loss: 60.28145 | Acc: 0.848
Epoch 09: | Loss: 64.08510 | Acc: 0.844
Epoch 10: | Loss: 61.82484 | Acc: 0.843
